In [1]:
import numpy as np
import math
from scipy.stats import unitary_group
from skimage.measure import block_reduce
from opt_einsum import contract
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
import torch
import time
import sys
import matplotlib.pyplot as plt
from matplotlib import rc
from src.QDDPM_torch import DiffusionModel, QDDPM, naturalDistance
#import src.ImageEncode as ie
rc('text', usetex=False)
rc('axes', linewidth=3)

Please first ``pip install -U qiskit`` to enable related functionality in translation module


In [10]:
def angle_encode(image: np.array):
    Npixels = np.size(image)

    image = image.flatten()

    #if ((Npixels & (Npixels - 1)) == 0 and Npixels != 0):
    #    print("Pixel number is not a power of 2")
    #    return -1

    states = (1 + 1j) * np.zeros(2 ** Npixels)
    scale_val = (0.5 * math.pi / np.max(image))
    image = image * scale_val

    for i in range(0, 2 ** Npixels):
        states[i] = 1 + 0j
        temp = i
        for j in range(0, Npixels):
            if (temp % 2):
                states[i] *= math.cos(image[Npixels - 1 - j])
            else:
                states[i] *= math.sin(image[Npixels - 1 - j])
            temp = int(temp / 2)
    
    return scale_val, states

In [15]:
#test angle_encode
temp_test = np.array([1, 1])
#plt.imshow(temp_test, cmap='grey',interpolation = 'nearest')
scale_val, states = angle_encode(temp_test)
print(states)

[1.00000000e+00+0.j 6.12323400e-17+0.j 6.12323400e-17+0.j
 3.74939946e-33+0.j]
